In [1]:
import pandas as pd

In [2]:
in_file = "./data/WPDX - GHA - WPDX Plus.csv"

In [3]:
df = pd.read_csv(in_file)
df.shape

/var/folders/pt/b5mvkwrj3z95dnjwxq9p91340000gn/T/ipykernel_12762/761013206.py:1: DtypeWarning: Columns (20,26,60,63) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(in_file)


(24223, 73)

In [4]:
df.head(3)

,row_id,#source,#lat_deg,#lon_deg,#report_date,#status_id,#water_source_clean,#water_source_category,#water_tech_clean,#water_tech_category,...,#subjective_quality,New Georeferenced Column,lat_lon_deg,public_data_source,converted,count,created_timestamp,updated_timestamp,is_duplicate,dataset_title
0,490491,Water4,9.752869,-2.494573,05/30/2024 12:00:00 AM,Yes,Piped Water,Piped Water,NaN,NaN,...,NaN,POINT (-2.4945725999999997 9.752869499999997),"(9.752869499999997°, -2.4945725999999997°)",https://catalog.waterpointdata.org/datafiles/w...,NaN,1,03/18/2022 06:23:12 PM,03/18/2022 06:23:12 PM,NaN,Water4_Global_2021-2022
1,644474,Living Water International,7.066837,-2.783900,07/26/2015 12:00:00 AM,Yes,Borehole/Tubewell,Well,Hand Pump - Afridev,Hand Pump,...,NaN,POINT (-2.7839 7.066836667),"(7.066836667°, -2.7839°)",https://catalog.waterpointdata.org/datafiles/t...,NaN,1,07/21/2020 08:01:34 PM,07/21/2020 08:01:34 PM,NaN,Living Water International_Africa Latin Americ...
2,13068,CRS,10.345544,-1.031766,03/22/2021 12:00:00 AM,Yes,Borehole/Tubewell,Well,Hand Pump,Hand Pump,...,NaN,POINT (-1.031766 10.345544),"(10.345544°, -1.031766°)",https://catalog.waterpointdata.org/datafiles/c...,NaN,1,02/15/2021 04:51:31 PM,02/15/2021 04:51:31 PM,NaN,Catholic Relief Services_Ghana_2021


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24223 entries, 0 to 24222
Data columns (total 73 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   row_id                       24223 non-null  int64  
 1   #source                      24223 non-null  object 
 2   #lat_deg                     24223 non-null  float64
 3   #lon_deg                     24223 non-null  float64
 4   #report_date                 24223 non-null  object 
 5   #status_id                   24223 non-null  object 
 6   #water_source_clean          23950 non-null  object 
 7   #water_source_category       23950 non-null  object 
 8   #water_tech_clean            23422 non-null  object 
 9   #water_tech_category         23280 non-null  object 
 10  #facility_type               24223 non-null  object 
 11  #clean_country_name          24223 non-null  object 
 12  #clean_country_id            24223 non-null  object 
 13  #clean_adm1     

In [6]:
df["#clean_adm1"].value_counts()

#clean_adm1
Upper West       4240
Upper East       4017
Western          2919
Central          2615
Northern         2604
Bono             1505
Savannah         1384
Bono East        1313
Western North    1194
Northern East    1138
Ahafo            1066
Oti               136
Ashanti            88
Eastern             3
Greater Accra       1
Name: count, dtype: int64

In [7]:
df["#source"].value_counts()

#source
CWSA                                            19610
Water4                                           3052
World Vision                                      457
Aquaya                                            276
CRS                                               270
Water4 Headquarters                               237
Living Water International                        127
Global Environment and Technology Foundation       74
Water4 Public                                      70
YouthMappers Ghana Chapter                         40
WaterAid                                            6
UNHCR                                               3
WaterAid UK                                         1
Name: count, dtype: int64

## Filter to piped water only

In [8]:
df["#water_source_clean"].value_counts()

#water_source_clean
Borehole/Tubewell          18675
Piped Water                 3016
Protected Well              2221
Undefined Well                19
Rainwater Harvesting          12
Sand or Sub-surface Dam        4
Undefined Spring               2
Protected Spring               1
Name: count, dtype: int64

In [9]:
df.shape

(24223, 73)

In [10]:
df = df[df["#water_source_clean"].isin(["Piped Water"])].copy()
df.shape

(3016, 73)

## Data Cleanup

EG: `systems_gdf.loc[systems_gdf["OPERATOR"] == "ristome (user)", "OPERATOR"] = "Unknown"`

In [11]:
df.loc[df["#source"]=="Water4 Headquarters", "#source"] = "Water4"
df.loc[df["#source"]=="Water4 Public", "#source"] = "Water4"
df.loc[df["#source"]=="WaterAid UK", "#source"] = "WaterAid"
df.loc[df["#management_clean"]=="Private Operator/Delegated Management", "#management_clean"] = "Private Operator/Delegated Management (WPDX)"

In [12]:
# Add Source
df["SOURCE"] = "Public-wpdx"
df["COMMUNITY"] = "Unknown-wpdx"

In [13]:
# Trim down columns
keep_cols = [
    '#source', '#lat_deg', '#lon_deg', '#report_date',
    '#clean_country_name', '#clean_country_id', '#clean_adm1', '#clean_adm2',
    '#water_source_clean', '#install_year', '#management_clean',
    '#distance_to_primary_road', '#distance_to_secondary_road', '#distance_to_tertiary_road', '#distance_to_city', '#distance_to_town',
    'usage_capacity', 'days_since_report', 'created_timestamp', 'updated_timestamp', "SOURCE", "COMMUNITY"
]

df = df[keep_cols].copy()
df.shape

(3016, 22)

In [14]:
# Clean up column names
rename_dict = {
    "#lat_deg": "LATITUDE",
    "#lon_deg": "LONGITUDE",
    "#clean_country_name": "COUNTRY",
    "#clean_adm1": "REGION",
    "#clean_adm2": "DISTRICT",
    "#management_clean": "OPERATOR",
    "#source": "#data_source", 
}

In [15]:
df = df.rename(columns=rename_dict)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3016 entries, 0 to 5378
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   #data_source                 3016 non-null   object 
 1   LATITUDE                     3016 non-null   float64
 2   LONGITUDE                    3016 non-null   float64
 3   #report_date                 3016 non-null   object 
 4   COUNTRY                      3016 non-null   object 
 5   #clean_country_id            3016 non-null   object 
 6   REGION                       3016 non-null   object 
 7   DISTRICT                     3016 non-null   object 
 8   #water_source_clean          3016 non-null   object 
 9   #install_year                2705 non-null   float64
 10  OPERATOR                     2770 non-null   object 
 11  #distance_to_primary_road    3016 non-null   float64
 12  #distance_to_secondary_road  3016 non-null   float64
 13  #distance_to_tertiary_r

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3016 entries, 0 to 5378
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   #data_source                 3016 non-null   object 
 1   LATITUDE                     3016 non-null   float64
 2   LONGITUDE                    3016 non-null   float64
 3   #report_date                 3016 non-null   object 
 4   COUNTRY                      3016 non-null   object 
 5   #clean_country_id            3016 non-null   object 
 6   REGION                       3016 non-null   object 
 7   DISTRICT                     3016 non-null   object 
 8   #water_source_clean          3016 non-null   object 
 9   #install_year                2705 non-null   float64
 10  OPERATOR                     2770 non-null   object 
 11  #distance_to_primary_road    3016 non-null   float64
 12  #distance_to_secondary_road  3016 non-null   float64
 13  #distance_to_tertiary_r

In [17]:
df.head(3)

,#data_source,LATITUDE,LONGITUDE,#report_date,COUNTRY,#clean_country_id,REGION,DISTRICT,#water_source_clean,#install_year,...,#distance_to_secondary_road,#distance_to_tertiary_road,#distance_to_city,#distance_to_town,usage_capacity,days_since_report,created_timestamp,updated_timestamp,SOURCE,COMMUNITY
0,Water4,9.752869,-2.494573,05/30/2024 12:00:00 AM,Ghana,GHA,Upper West,Wa West,Piped Water,2024.0,...,48722.487507,20068.027173,34248.810205,22288.223218,250.0,17,03/18/2022 06:23:12 PM,03/18/2022 06:23:12 PM,Public-wpdx,Unknown-wpdx
14,Water4,9.492992,-2.417155,11/15/2023 12:00:00 AM,Ghana,GHA,Savannah,Sawla-Tuna-Kalba,Piped Water,2023.0,...,69669.970007,276.649434,63712.378702,24362.625525,250.0,214,03/18/2022 06:23:12 PM,03/18/2022 06:23:12 PM,Public-wpdx,Unknown-wpdx
15,Water4,5.353420,-1.675617,06/21/2023 12:00:00 AM,Ghana,GHA,Western,Wassa East,Piped Water,NaN,...,69.403511,8921.660539,47802.405045,23164.055631,250.0,361,05/19/2021 07:21:29 PM,05/19/2021 07:21:29 PM,Public-wpdx,Unknown-wpdx


## Export

In [18]:
df.to_excel("./export/AFPW-PipedWaterSystems-WPDX.xlsx", index=False)